<a href="https://colab.research.google.com/github/AbhiAbhiraj621/Smart_City_Optimization_Using_PySpark/blob/main/Smart_City_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.csv('/content/sample_data/smart_city_energy.csv', header = True, inferSchema = True)
df.show()

+-------+-------------------+---------------+-------------+----------------+
|zone_id|          timestamp|consumption_kwh|temperature_c|humidity_percent|
+-------+-------------------+---------------+-------------+----------------+
| Zone_1|2025-01-01 00:00:00|           3.08|         24.4|            56.4|
| Zone_2|2025-01-01 00:00:00|           1.91|         31.4|            52.0|
| Zone_3|2025-01-01 00:00:00|           2.05|         22.6|            48.8|
| Zone_4|2025-01-01 00:00:00|           2.01|         27.3|            62.7|
| Zone_5|2025-01-01 00:00:00|            1.5|         23.6|            56.5|
| Zone_6|2025-01-01 00:00:00|           2.51|         24.2|            59.6|
| Zone_7|2025-01-01 00:00:00|           1.02|         32.0|            66.5|
| Zone_8|2025-01-01 00:00:00|           1.49|         28.6|            75.5|
| Zone_9|2025-01-01 00:00:00|           1.53|         22.5|            62.4|
|Zone_10|2025-01-01 00:00:00|           1.57|         21.6|            58.0|

In [3]:
df.printSchema()

root
 |-- zone_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- consumption_kwh: double (nullable = true)
 |-- temperature_c: double (nullable = true)
 |-- humidity_percent: double (nullable = true)



In [10]:
# Handling missing values
df = df.na.drop()
df = df.filter(df['consumption_kwh'] > 0)

In [14]:
# Extracting hour and date
df = df.withColumn('date', to_date('timestamp'))
df = df.withColumn('hour',hour('timestamp'))

In [16]:
# What is the total consumption per zone
df.groupBy('zone_id').sum('consumption_kwh').orderBy('sum(consumption_kwh)',ascending=False).show()

+-------+--------------------+
|zone_id|sum(consumption_kwh)|
+-------+--------------------+
| Zone_2|  2157.1600000000008|
|Zone_10|   2153.519999999998|
| Zone_3|   2144.750000000003|
| Zone_1|             2144.21|
| Zone_9|  2136.9399999999982|
| Zone_4|   2130.450000000001|
| Zone_6|  2129.6800000000003|
| Zone_8|  2125.2400000000016|
| Zone_7|             2124.48|
| Zone_5|  2124.3900000000044|
+-------+--------------------+



In [17]:
# What is the average hourly consumption
df.groupBy('hour').avg('consumption_kwh').orderBy('hour').show()

+----+--------------------+
|hour|avg(consumption_kwh)|
+----+--------------------+
|   0|  1.9664999999999995|
|   1|   1.993366666666667|
|   2|  2.0501333333333323|
|   3|   2.014247491638797|
|   4|   2.000633333333334|
|   5|  1.9138000000000022|
|   6|   2.027166666666666|
|   7|  3.0364999999999975|
|   8|   3.000033333333333|
|   9|  2.9562333333333353|
|  10|  3.0318333333333336|
|  11|  3.0034666666666667|
|  12|  3.0085333333333324|
|  13|              3.0407|
|  14|   3.016199999999998|
|  15|  3.0098999999999996|
|  16|  2.9806333333333326|
|  17|   3.002266666666668|
|  18|  4.0442333333333345|
|  19|    4.06026666666667|
+----+--------------------+
only showing top 20 rows



In [19]:
# Is there any correlation with the weather
df.select('consumption_kwh','temperature_c','humidity_percent').show()

+---------------+-------------+----------------+
|consumption_kwh|temperature_c|humidity_percent|
+---------------+-------------+----------------+
|           3.08|         24.4|            56.4|
|           1.91|         31.4|            52.0|
|           2.05|         22.6|            48.8|
|           2.01|         27.3|            62.7|
|            1.5|         23.6|            56.5|
|           2.51|         24.2|            59.6|
|           1.02|         32.0|            66.5|
|           1.49|         28.6|            75.5|
|           1.53|         22.5|            62.4|
|           1.57|         21.6|            58.0|
|           2.54|         19.2|            63.1|
|           1.81|         27.5|            54.0|
|           1.68|         16.0|            48.7|
|           2.12|         27.0|            58.3|
|           2.24|         26.5|            64.0|
|           2.69|         17.1|            49.2|
|           1.87|         25.1|            48.3|
|           1.11|   

In [20]:
# When is the peak usage for each zone
from pyspark.sql.window import Window

peak_usage = df.groupBy('zone_id','hour').agg(avg('consumption_kwh').alias('avg_consumption')).orderBy('zone_id','avg_consumption',ascending=False)

window_spec = Window.partitionBy('zone_id').orderBy(peak_usage['avg_consumption'].desc())
peak_hour_per_zone = peak_usage.withColumn('rank', row_number().over(window_spec)).filter('rank = 1').select('zone_id','hour','avg_consumption')
peak_hour_per_zone.show()

+-------+----+------------------+
|zone_id|hour|   avg_consumption|
+-------+----+------------------+
| Zone_1|  22|4.1386666666666665|
|Zone_10|  21|              4.21|
| Zone_2|  19| 4.184999999999999|
| Zone_3|  19| 4.138333333333333|
| Zone_4|  20| 4.102666666666667|
| Zone_5|  19|4.2490000000000006|
| Zone_6|  20|             4.175|
| Zone_7|  23|              4.18|
| Zone_8|  20|4.1339999999999995|
| Zone_9|  19|             4.121|
+-------+----+------------------+



In [21]:
# How temperature affects usage
correlation = df.stat.corr('temperature_c','consumption_kwh')
print(correlation)

0.036882934312364245


In [23]:
# Which are the zone with highest average peak-hour consumption
peak_zone_consumption = df.filter('hour >= 18 AND hour <= 22').groupBy('zone_id').avg('consumption_kwh').orderBy('avg(consumption_kwh)',ascending=False)
peak_zone_consumption.show()

+-------+--------------------+
|zone_id|avg(consumption_kwh)|
+-------+--------------------+
|Zone_10|   4.108666666666669|
| Zone_5|   4.053933333333335|
| Zone_3|   4.047933333333336|
| Zone_8|   4.042399999999996|
| Zone_1|   4.034333333333333|
| Zone_9|   4.029066666666667|
| Zone_2|   4.021866666666666|
| Zone_7|              4.0022|
| Zone_6|  3.9963999999999995|
| Zone_4|   3.956799999999999|
+-------+--------------------+

